> # Effective Localization via Attentive Learning  Chest X-ray 
> IAI - Team DA - Dayoung Lee           

#      
#      
#     
## 1. **Data I/O of Bounding Box and corresponding Chest X-ray Images.**    

#     
---

#     
#        
### 1-1.  Mounting Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')
!ls -l

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/
total 8
drwx------ 4 root root 4096 Apr  4 03:18 drive
drwxr-xr-x 1 root root 4096 Mar 18 16:23 sample_data


In [ ]:
# OS
!printf "\n------- OS -------\n"
!cat /etc/issue.net
# CPU 사양
!printf "\n---- CPU info. ----\n"
!head /proc/cpuinfo
# 메모리 사양
!printf "\n---- Mem info. ----\n"
!head -n 3 /proc/meminfo
# 디스크 사양
!printf "\n---- Disc info. ---\n"
!df -h


------- OS -------
Ubuntu 18.04.3 LTS

---- CPU info. ----
processor	: 0
vendor_id	: GenuineIntel
cpu family	: 6
model		: 63
model name	: Intel(R) Xeon(R) CPU @ 2.30GHz
stepping	: 0
microcode	: 0x1
cpu MHz		: 2300.000
cache size	: 46080 KB
physical id	: 0

---- Mem info. ----
MemTotal:       13333556 kB
MemFree:        10515064 kB
MemAvailable:   12503752 kB

---- Disc info. ---
Filesystem      Size  Used Avail Use% Mounted on
overlay         108G   31G   72G  31% /
tmpfs            64M     0   64M   0% /dev
tmpfs           6.4G     0  6.4G   0% /sys/fs/cgroup
shm             5.9G     0  5.9G   0% /dev/shm
tmpfs           6.4G   12K  6.4G   1% /var/colab
/dev/sda1       114G   33G   82G  29% /etc/hosts
tmpfs           6.4G     0  6.4G   0% /proc/acpi
tmpfs           6.4G     0  6.4G   0% /proc/scsi
tmpfs           6.4G     0  6.4G   0% /sys/firmware


#    
### 1-2.  Configuration Setting

In [ ]:
!python --version

import os,functools
os.environ["KERAS_BACKEND"] = "plaidml.keras.backend"

from os.path import join, exists
from glob import glob 
import numpy as np
import pandas as pd
from collections import OrderedDict, Counter

import cv2
import PIL


from sklearn.preprocessing import MultiLabelBinarizer, OneHotEncoder, LabelEncoder

Python 3.6.9


In [ ]:
os.chdir('/content/drive/My Drive/IAI_Korea_AI_Institute')#/Dataset')
root = os.getcwd()
print(root)
!ls

In [ ]:
source = os.getcwd()
# DATASET_DIR = glob('Dataset/images_*/')
LABEL_FILE = os.path.join(source,'Dataset/BBox_List_2017.csv')  # 'BBox_List_2017.csv')
DATASET_DIR = join(os.getcwd(), 'Dataset/')  #'_dataset_sample/')
# LABEL_FILE = 'BBox_List_2017.csv'
# print(os.listdir(dataset_dir))

HEIGHT, WIDTH = 512, 512

#    
### 1-3. Dataset I/O

In [ ]:
def pause():
    input('\n\n Press the <Enter> key to continue...\n')
    return None


def print_1by1(target_list):
    for i, row in enumerate(target_list):
        print(row)


def mkdir_ifnot(path):
    try:
        if not os.path.exists(path):
            os.mkdir(path)
    except OSError as e:
        print(e.errno)
        print("DY: Failed to create directory, %s" % path)
        raise
    return path


def makedirs_ifnot(path):
    try:
        if not os.path.exists(path):
            os.makedirs(os.path.join(path))
        else:
            print(path, " is already exist.\n")
    except OSError as e:
        print(e.errno)
        print("DY: Failed to create directories, %s" % path)
        raise
    return path

In [ ]:
"""
 Getting Image Information
"""

def get_img_path_list(dataset_dir):
    ##### Get image path
    img_path_list = []
    for (Par, Subs, Files) in os.walk(dataset_dir):
        for file in Files:
            ext = os.path.splitext(file)[-1]
            if ext in ['.JPEG', '.JPG', '.PNG', '.jpeg', '.jpg', '.png']:
                img_path = os.path.join(Par, file)
                img_path_list.append(img_path)
    print('>>> img_path_list : ',len(img_path_list))
    print_1by1(img_path_list[0:3])
    print('\n')
    return img_path_list



"""
 Getting Excel Information FOR Bbox_List
"""

def get_exel_info(label_file):
    with open(label_file, mode='r', encoding='utf-8') as txt_file:
      txt_file_lines = txt_file.readlines()

    excel_info = []
    for each_line in txt_file_lines:
      tmp1 = each_line.split(',') # '\t'
      tmp2 = list(map(lambda s: s.strip(' \n'), tmp1))
      excel_info.append(tmp2)
    main_excel_info = excel_info[1:]
    print('>>> main_excel_info: ', len(main_excel_info))
    print_1by1(main_excel_info[:3])
    print('\n')

    ##### Extract the distinct file names and info where the count is only one.
    c = Counter([line[0] for i, line in enumerate(main_excel_info)])    
    distinct_f_names = [ele for ele in c.elements() if c[ele]==1]
    # distinct_f_names = list(OrderedDict.fromkeys([line[0] for i, line in enumerate(main_excel_info)]))
    print('>>> distinct_f_names: ', len(distinct_f_names))
    distinct_excel_info = [line for i, line in enumerate(main_excel_info) if line[0] in distinct_f_names]
    print('>>> distinct_excel_info: ', len(distinct_excel_info))
    print_1by1(distinct_excel_info[:3])
    print('\n')

    ##### Convert labels from strings into binary with scikit.preprocessing.
    mlb = LabelEncoder()
    labels = [line[1] for line in distinct_excel_info]
    labels_ = mlb.fit_transform(labels)
    print('>>> labels_: ', len(labels_))

    ##### ['Image Index', 'Finding Labels', 'Bbox [x,y,w,h]']
    file_info_list_ = []  ##### 
    for i, line in enumerate(distinct_excel_info):
      file_info_list_.append([line[0], labels_[i], line[2:]])
    print('>>> file_info: ', len(file_info_list_))
    print_1by1(file_info_list_[:3])
    print('\n')

    return file_info_list_


img_path_list = get_img_path_list(DATASET_DIR)
file_info_list = get_exel_info(LABEL_FILE)

>>> img_path_list :  112342
/content/drive/My Drive/IAI_Korea_AI_Institute/Dataset/images_001/images/00001301_001.png
/content/drive/My Drive/IAI_Korea_AI_Institute/Dataset/images_001/images/00001301_000.png
/content/drive/My Drive/IAI_Korea_AI_Institute/Dataset/images_001/images/00001301_002.png


>>> main_excel_info:  984
['00013118_008.png', 'Atelectasis', '225.084745762712', '547.019216763771', '86.7796610169491', '79.1864406779661']
['00014716_007.png', 'Atelectasis', '686.101694915254', '131.543498411017', '185.491525423729', '313.491525423729']
['00029817_009.png', 'Atelectasis', '221.830508474576', '317.053115068856', '155.118644067797', '216.949152542373']


>>> distinct_f_names:  787
>>> distinct_excel_info:  787
['00013118_008.png', 'Atelectasis', '225.084745762712', '547.019216763771', '86.7796610169491', '79.1864406779661']
['00014716_007.png', 'Atelectasis', '686.101694915254', '131.543498411017', '185.491525423729', '313.491525423729']
['00029817_009.png', 'Atelectasis',

In [ ]:
"""
 Matching Image path and Patient Info.
"""

def match_pathNinfo(file_info_list, img_path_list):
  matching_list, unmatched_list = [], []
  # for idx, each_dir in enumerate(img_path_list):
  for idx, value in enumerate(file_info_list):
    file_name = value[0]
    label = value[1]
    bbox = value[2]
    matched = 0
    for each_dir in img_path_list:  # 106116
      file_name_from_dir1 = os.path.split(each_dir)[-1]
      if file_name in each_dir:
        matching_list.append([file_name, label, bbox, each_dir])
        matched = 1
        break
      else:
        pass
      # # 매칭 안된 이미지들 확인용.
      # if matched != 1:
      #     unmatched_list.append([file_name])


  img_name_list = [val[0] for val in matching_list]
  img_dir_list = [val[-1] for val in matching_list]
  label_list = [val[1] for val in matching_list]
  bbox_list = [val[2] for val in matching_list]

  print('>>> matching_list : ', len(matching_list))
  print(matching_list[0:3],',  ...')
  # print('>>> img_dir_list : ', len(img_dir_list))
  # print_1by1(img_dir_list[:3])
  # print('>>> label_list : ', len(label_list))
  # print_1by1(label_list[:3])
  print('\r'+'-'*70)
  # print('>>> unmatched_list : ', len(unmatched_list))

  return img_name_list, img_dir_list, label_list

img_name_list, img_dir_list, label_list = match_pathNinfo(file_info_list, img_path_list)

>>> matching_list :  787
[['00013118_008.png', 0, ['225.084745762712', '547.019216763771', '86.7796610169491', '79.1864406779661'], '/content/drive/My Drive/IAI_Korea_AI_Institute/Dataset/images_006/images/00013118_008.png'], ['00014716_007.png', 0, ['686.101694915254', '131.543498411017', '185.491525423729', '313.491525423729'], '/content/drive/My Drive/IAI_Korea_AI_Institute/Dataset/images_007/images/00014716_007.png'], ['00029817_009.png', 0, ['221.830508474576', '317.053115068856', '155.118644067797', '216.949152542373'], '/content/drive/My Drive/IAI_Korea_AI_Institute/Dataset/images_012/images/00029817_009.png']] ,  ...
----------------------------------------------------------------------


#    
### 1-4. Pre-processing (Generating B-Box and Resizing)

In [ ]:
"""
 Loading Pickle Files
"""

def load_pickle(path):
    with open(path, 'rb') as f:
        load_pickle = pickle.load(f)

        np_array_values = load_pickle[0:-1]
        train_array = np_array_values[0]
        real_train_array = np.array(train_array)
        # real_train_array = np.repeat(real_train_array[:, :, :, np.newaxis], 3, -1)

        get_label = load_pickle[-1]
        label_no_cat = np.array(get_label)
        label = np_utils.to_categorical(get_label, 2)
        real_train_label = np.array(label)

        print("\n")
        print("===========[ Loading Pickle File ]===========")
        # print('   >>> loaded train shape: ', load_pickle.shape)
        # print('   >>> loaded label shape: ', load_pickle.shape)
        print("\n")
        print("   >>> real_train_array", real_train_array.shape)
        print("   >>> real_train_label", real_train_label.shape)
        print("=============================================")
        print("\n")

    return real_train_array, real_train_label, label_no_cat
